In [7]:
%pip install wordcloud nltk seaborn

Note: you may need to restart the kernel to use updated packages.


In [8]:
"""
Análisis Exploratorio de Datos (EDA) para la Base de Conocimiento
================================================================

Este notebook realiza un análisis exhaustivo de la base de conocimiento utilizada
en el sistema de asistente de ventas consultivas. Proporciona insights estadísticos
y visualizaciones que justifican las decisiones de diseño del sistema.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import spacy
import pickle
import faiss
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
def load_and_configure_data():
    """
    Carga y configura los datos necesarios para el análisis exploratorio.
    
    Esta función establece la configuración visual para los gráficos y carga
    los datasets principales del sistema de ventas consultivas.
    
    Returns:
        tuple: (sentencias_df, mapping_df, nlp) - DataFrames y modelo de spaCy
        
    Justificación técnica:
        - seaborn-v0_8: Estilo visual profesional para publicaciones académicas
        - husl palette: Colores distintivos y accesibles para visualizaciones
        - spaCy es_core_news_sm: Modelo preentrenado optimizado para español
    """
    # Configuración de visualización para presentaciones académicas
    plt.style.use('seaborn-v0_8')
    sns.set_palette("husl")
    
    # Cargar datasets principales
    print("Cargando los datos...")
    sentencias_df = pd.read_csv('sentencias.csv')
    mapping_df = pd.read_csv('mapping.csv')
    
    # Cargar modelo de procesamiento de lenguaje natural para español
    nlp = spacy.load("es_core_news_sm")
    
    return sentencias_df, mapping_df, nlp

# Ejecutar configuración y carga de datos
sentencias_df, mapping_df, nlp = load_and_configure_data()


Cargando los datos...


In [10]:
def perform_comprehensive_eda(sentencias_df):
    """
    Realiza un análisis exploratorio completo de la base de conocimiento.
    
    Esta función ejecuta múltiples análisis estadísticos y genera visualizaciones
    para caracterizar la base de conocimiento del sistema de ventas consultivas.
    
    Args:
        sentencias_df (pd.DataFrame): DataFrame con las sentencias del playbook
        
    Returns:
        dict: Diccionario con métricas y estadísticas calculadas
        
    Funcionalidades:
        1. Análisis descriptivo básico (conteos, distribuciones)
        2. Análisis de longitud de sentencias (estadísticas descriptivas)
        3. Análisis de frecuencia de palabras (con filtrado de stopwords)
        4. Generación de visualizaciones (histogramas, wordclouds, gráficos de barras)
        5. Análisis comparativo por documento fuente
    """
    
    def preprocess_text(text):
        """
        Preprocesa texto para análisis de frecuencia de palabras.
        
        Args:
            text (str): Texto a procesar
            
        Returns:
            str: Texto limpio en minúsculas sin caracteres especiales
        """
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        return text
    
    print("\n=== Análisis Básico de las Sentencias ===")
    print(f"Número total de sentencias: {len(sentencias_df)}")
    print(f"Número de documentos únicos: {sentencias_df['path'].nunique()}")
    
    # Análisis de longitud de sentencias - crítico para configurar parámetros FAISS
    sentencias_df['longitud'] = sentencias_df['sentencias'].str.len()
    print("\n=== Estadísticas de Longitud de Sentencias ===")
    print(sentencias_df['longitud'].describe())
    
    # Visualización de distribución de longitudes
    plt.figure(figsize=(10, 6))
    sns.histplot(data=sentencias_df, x='longitud', bins=50, alpha=0.7)
    plt.title('Distribución de Longitud de Sentencias\n(Fundamental para configurar límites de fragmentos)')
    plt.xlabel('Longitud (caracteres)')
    plt.ylabel('Frecuencia')
    plt.axvline(sentencias_df['longitud'].mean(), color='red', linestyle='--', 
                label=f'Media: {sentencias_df["longitud"].mean():.1f}')
    plt.legend()
    plt.savefig('distribucion_longitud.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Análisis de frecuencia de palabras
    all_words = ' '.join(sentencias_df['sentencias'].apply(preprocess_text))
    words = all_words.split()
    stop_words = set(stopwords.words('spanish'))
    # Filtrar palabras cortas y stopwords para análisis más significativo
    filtered_words = [word for word in words if word not in stop_words and len(word) > 3]
    word_freq = Counter(filtered_words).most_common(20)
    
    # Visualización de palabras más frecuentes
    plt.figure(figsize=(12, 6))
    words_list, freqs = zip(*word_freq)
    bars = plt.bar(words_list, freqs, alpha=0.8)
    plt.xticks(rotation=45, ha='right')
    plt.title('20 Palabras Más Comunes en la Base de Conocimiento\n(Después de filtrar stopwords)')
    plt.xlabel('Palabras')
    plt.ylabel('Frecuencia')
    plt.tight_layout()
    
    # Agregar valores en las barras para mejor legibilidad
    for bar, freq in zip(bars, freqs):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                str(freq), ha='center', va='bottom', fontsize=8)
    
    plt.savefig('palabras_comunes.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Generación de WordCloud para visualización semántica
    wordcloud = WordCloud(
        width=800, height=400, 
        background_color='white', 
        max_words=100,
        colormap='viridis',
        relative_scaling=0.5
    ).generate(all_words)
    
    plt.figure(figsize=(15, 8))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('WordCloud de la Base de Conocimiento\n(Visualización semántica del corpus)', 
              fontsize=16, pad=20)
    plt.savefig('wordcloud.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Análisis comparativo por documento fuente
    print("\n=== Análisis por Documento ===")
    doc_stats = sentencias_df.groupby('path').agg({
        'sentencias': 'count',
        'longitud': ['mean', 'std', 'min', 'max']
    }).round(2)
    print(doc_stats)
    
    # Exportar estadísticas para documentación
    doc_stats.to_csv('estadisticas_documentos.csv')
    
    # Calcular métricas adicionales para retorno
    metrics = {
        'total_sentences': len(sentencias_df),
        'unique_documents': sentencias_df['path'].nunique(),
        'avg_length': sentencias_df['longitud'].mean(),
        'std_length': sentencias_df['longitud'].std(),
        'total_words': len(words),
        'unique_words': len(set(filtered_words)),
        'vocab_richness': len(set(filtered_words)) / len(filtered_words) if filtered_words else 0
    }
    
    print(f"\n=== Métricas Adicionales ===")
    print(f"Riqueza del vocabulario: {metrics['vocab_richness']:.3f}")
    print(f"Palabras únicas (filtradas): {metrics['unique_words']}")
    
    print("\nAnálisis completado. Archivos generados:")
    print("- distribucion_longitud.png (Histograma de longitudes)")
    print("- palabras_comunes.png (Top 20 palabras más frecuentes)")
    print("- wordcloud.png (Visualización semántica)")
    print("- estadisticas_documentos.csv (Métricas por documento)")
    
    return metrics

# Ejecutar análisis exploratorio completo
eda_metrics = perform_comprehensive_eda(sentencias_df) 


=== Análisis Básico de las Sentencias ===
Número total de sentencias: 587
Número de documentos únicos: 8

=== Estadísticas de Longitud de Sentencias ===
count     587.000000
mean      193.206133
std       292.285329
min         8.000000
25%        68.000000
50%       131.000000
75%       213.000000
max      2948.000000
Name: longitud, dtype: float64

=== Análisis por Documento ===
                                                   sentencias longitud  \
                                                        count     mean   
path                                                                     
Base de datos para herramienta\promt.txt                   94   156.05   
Base de datos para herramienta\txt_extraidos\Ej...         58   593.62   
Base de datos para herramienta\txt_extraidos\Ev...         35   115.23   
Base de datos para herramienta\txt_extraidos\PR...          4   259.00   
Base de datos para herramienta\txt_extraidos\Pl...         49   170.20   
Base de datos para herr

In [11]:
def perform_advanced_eda_analysis(sentencias_df, mapping_df):
    """
    Análisis exploratorio avanzado con métricas adicionales y visualizaciones mejoradas.
    
    Esta función complementa el EDA básico con análisis más profundos que proporcionan
    insights adicionales sobre la calidad y características de la base de conocimiento.
    
    Args:
        sentencias_df (pd.DataFrame): DataFrame con las sentencias del playbook
        mapping_df (pd.DataFrame): DataFrame con el mapeo de índices
        
    Returns:
        dict: Métricas avanzadas calculadas
        
    Mejoras implementadas:
        1. Análisis de diversidad léxica por documento
        2. Detección de duplicados y similitudes
        3. Análisis de complejidad sintáctica
        4. Correlaciones entre variables
        5. Análisis de outliers
        6. Métricas de calidad del corpus
        
    """
    
    print("\n=== ANÁLISIS EXPLORATORIO AVANZADO ===")
    
    # 1. ANÁLISIS DE DUPLICADOS Y SIMILITUDES
    print("\n1. Análisis de Duplicados y Similitudes:")
    duplicates = sentencias_df['sentencias'].duplicated().sum()
    duplicate_percentage = (duplicates / len(sentencias_df)) * 100
    print(f"   - Sentencias duplicadas exactas: {duplicates} ({duplicate_percentage:.2f}%)")
    
    # Detectar sentencias muy cortas (posibles outliers)
    very_short = sentencias_df[sentencias_df['longitud'] <= 10]
    print(f"   - Sentencias muy cortas (≤10 chars): {len(very_short)} ({len(very_short)/len(sentencias_df)*100:.2f}%)")
    
    # Detectar sentencias muy largas (posibles outliers)
    very_long = sentencias_df[sentencias_df['longitud'] >= 500]
    print(f"   - Sentencias muy largas (≥500 chars): {len(very_long)} ({len(very_long)/len(sentencias_df)*100:.2f}%)")
    
    # 2. ANÁLISIS DE DIVERSIDAD LÉXICA POR DOCUMENTO
    print("\n2. Diversidad Léxica por Documento:")
    lexical_diversity = {}
    
    for path in sentencias_df['path'].unique():
        doc_sentences = sentencias_df[sentencias_df['path'] == path]['sentencias']
        all_text = ' '.join(doc_sentences).lower()
        words = re.findall(r'\b\w+\b', all_text)
        unique_words = set(words)
        
        # Type-Token Ratio (TTR) - medida de diversidad léxica
        ttr = len(unique_words) / len(words) if words else 0
        lexical_diversity[path] = {
            'total_words': len(words),
            'unique_words': len(unique_words),
            'ttr': ttr
        }
        
        doc_name = path.split('\\')[-1] if '\\' in path else path
        print(f"   - {doc_name}: TTR = {ttr:.3f} ({len(unique_words)} palabras únicas de {len(words)} totales)")
    
    # 3. ANÁLISIS DE DISTRIBUCIÓN DE LONGITUDES POR DOCUMENTO
    print("\n3. Distribución de Longitudes por Documento:")
    
    plt.figure(figsize=(15, 10))
    
    # Subplot 1: Boxplot por documento
    plt.subplot(2, 2, 1)
    sentencias_df['doc_name'] = sentencias_df['path'].apply(lambda x: x.split('\\')[-1] if '\\' in x else x)
    sns.boxplot(data=sentencias_df, y='doc_name', x='longitud')
    plt.title('Distribución de Longitudes por Documento')
    plt.xlabel('Longitud (caracteres)')
    
    # Subplot 2: Histograma con curva de densidad
    plt.subplot(2, 2, 2)
    sns.histplot(data=sentencias_df, x='longitud', kde=True, bins=30)
    plt.axvline(sentencias_df['longitud'].median(), color='orange', linestyle='--', label='Mediana')
    plt.axvline(sentencias_df['longitud'].mean(), color='red', linestyle='--', label='Media')
    plt.title('Distribución con Curva de Densidad')
    plt.legend()
    
    # Subplot 3: Gráfico Q-Q para normalidad
    from scipy import stats
    plt.subplot(2, 2, 3)
    stats.probplot(sentencias_df['longitud'], dist="norm", plot=plt)
    plt.title('Q-Q Plot (Prueba de Normalidad)')
    plt.grid(True, alpha=0.3)
    
    # Subplot 4: Análisis de outliers
    plt.subplot(2, 2, 4)
    Q1 = sentencias_df['longitud'].quantile(0.25)
    Q3 = sentencias_df['longitud'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = sentencias_df[(sentencias_df['longitud'] < lower_bound) | 
                            (sentencias_df['longitud'] > upper_bound)]
    
    plt.scatter(range(len(sentencias_df)), sentencias_df['longitud'], alpha=0.6, s=20)
    plt.axhline(upper_bound, color='red', linestyle='--', label=f'Límite superior: {upper_bound:.0f}')
    plt.axhline(lower_bound, color='red', linestyle='--', label=f'Límite inferior: {lower_bound:.0f}')
    plt.title(f'Detección de Outliers ({len(outliers)} encontrados)')
    plt.xlabel('Índice de sentencia')
    plt.ylabel('Longitud')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('analisis_avanzado_longitudes.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"   - Outliers detectados: {len(outliers)} ({len(outliers)/len(sentencias_df)*100:.2f}%)")
    print(f"   - Rango intercuartílico (IQR): {IQR:.1f}")
    print(f"   - Límites para outliers: [{lower_bound:.1f}, {upper_bound:.1f}]")
    
    # 4. ANÁLISIS DE COMPLEJIDAD SINTÁCTICA
    print("\n4. Análisis de Complejidad Sintáctica:")
    
    # Calcular métricas de complejidad
    sentencias_df['num_words'] = sentencias_df['sentencias'].apply(lambda x: len(x.split()))
    sentencias_df['num_sentences'] = sentencias_df['sentencias'].apply(lambda x: len(re.split(r'[.!?]+', x)))
    sentencias_df['avg_word_length'] = sentencias_df['sentencias'].apply(
        lambda x: np.mean([len(word) for word in x.split()]) if x.split() else 0
    )
    
    complexity_stats = {
        'avg_words_per_sentence': sentencias_df['num_words'].mean(),
        'avg_word_length': sentencias_df['avg_word_length'].mean(),
        'lexical_density': sentencias_df['longitud'].mean() / sentencias_df['num_words'].mean()
    }
    
    print(f"   - Promedio de palabras por sentencia: {complexity_stats['avg_words_per_sentence']:.2f}")
    print(f"   - Longitud promedio de palabras: {complexity_stats['avg_word_length']:.2f} caracteres")
    print(f"   - Densidad léxica: {complexity_stats['lexical_density']:.2f}")
    
    # 5. MATRIZ DE CORRELACIONES
    print("\n5. Análisis de Correlaciones:")
    
    correlation_data = sentencias_df[['longitud', 'num_words', 'avg_word_length']].copy()
    correlation_matrix = correlation_data.corr()
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
                square=True, fmt='.3f')
    plt.title('Matriz de Correlaciones entre Variables')
    plt.tight_layout()
    plt.savefig('matriz_correlaciones.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print("   - Matriz de correlaciones guardada en 'matriz_correlaciones.png'")
    
    # 6. ANÁLISIS DE ENTROPÍA Y DIVERSIDAD
    print("\n6. Análisis de Entropía y Diversidad:")
    
    # Calcular entropía de Shannon para el corpus completo
    all_text = ' '.join(sentencias_df['sentencias']).lower()
    words = re.findall(r'\b\w+\b', all_text)
    word_freq = Counter(words)
    total_words = len(words)
    
    # Entropía de Shannon
    entropy = -sum((freq/total_words) * np.log2(freq/total_words) 
                   for freq in word_freq.values())
    
    print(f"   - Entropía de Shannon: {entropy:.3f} bits")
    print(f"   - Vocabulario total: {len(word_freq)} palabras únicas")
    print(f"   - Ratio de diversidad global: {len(word_freq)/total_words:.4f}")
    
    # 7. GENERAR REPORTE CONSOLIDADO
    def convert_numpy_types(obj):
        """Convierte tipos de numpy/pandas a tipos nativos de Python para serialización JSON"""
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, dict):
            return {key: convert_numpy_types(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [convert_numpy_types(item) for item in obj]
        else:
            return obj
    
    advanced_metrics = {
        'duplicates': int(duplicates),
        'duplicate_percentage': float(duplicate_percentage),
        'very_short_count': len(very_short),
        'very_long_count': len(very_long),
        'outliers_count': len(outliers),
        'outliers_percentage': float(len(outliers)/len(sentencias_df)*100),
        'shannon_entropy': float(entropy),
        'total_vocabulary': len(word_freq),
        'global_diversity_ratio': float(len(word_freq)/total_words),
        'lexical_diversity_by_doc': convert_numpy_types(lexical_diversity),
        'complexity_stats': convert_numpy_types(complexity_stats),
        'correlation_matrix': convert_numpy_types(correlation_matrix.round(3).to_dict())
    }
    
    # Guardar métricas avanzadas
    import json
    with open('metricas_avanzadas.json', 'w', encoding='utf-8') as f:
        json.dump(advanced_metrics, f, indent=2, ensure_ascii=False)
    
    print("\n=== ARCHIVOS GENERADOS ===")
    print("- analisis_avanzado_longitudes.png (Análisis multivariado de longitudes)")
    print("- matriz_correlaciones.png (Correlaciones entre variables)")
    print("- metricas_avanzadas.json (Todas las métricas calculadas)")
    
    return advanced_metrics

# Ejecutar análisis avanzado
print("Iniciando análisis exploratorio avanzado...")
advanced_results = perform_advanced_eda_analysis(sentencias_df, mapping_df)


Iniciando análisis exploratorio avanzado...

=== ANÁLISIS EXPLORATORIO AVANZADO ===

1. Análisis de Duplicados y Similitudes:
   - Sentencias duplicadas exactas: 1 (0.17%)
   - Sentencias muy cortas (≤10 chars): 4 (0.68%)
   - Sentencias muy largas (≥500 chars): 26 (4.43%)

2. Diversidad Léxica por Documento:
   - promt.txt: TTR = 0.332 (782 palabras únicas de 2352 totales)
   - Ejemplo de entregable al cliente de consulta a persona natural plan cumplimiento.txt: TTR = 0.151 (791 palabras únicas de 5235 totales)
   - Evaluación de la venta.txt: TTR = 0.517 (307 palabras únicas de 594 totales)
   - Plantilla propuesta cumplimiento.txt: TTR = 0.243 (486 palabras únicas de 1997 totales)
   - Plantilla propuesta validación.txt: TTR = 0.417 (363 palabras únicas de 871 totales)
   - Playbook de Evaluación (EVA) - Tusdatos.co.txt: TTR = 0.291 (1551 palabras únicas de 5327 totales)
   - PROPUESTA DE VALOR.txt: TTR = 0.497 (87 palabras únicas de 175 totales)
   - nuevo.txt: TTR = 0.667 (4 palab

## Conclusiones del Análisis Exploratorio Avanzado

### Hallazgos Principales del Análisis Básico

1. **Distribución de Longitudes**: La base de conocimiento contiene **587 sentencias** con una distribución asimétrica y media de **193.2 caracteres** por sentencia, justificando la configuración de límites de fragmentos en el sistema FAISS.

2. **Diversidad Documental**: Se identificaron **8 documentos fuente únicos**, proporcionando diversidad temática robusta en la base de conocimiento, desde prompts hasta plantillas especializadas.

3. **Vocabulario Especializado**: El análisis revela **2,534 palabras únicas filtradas** con una riqueza vocabular de **0.302**, validando la especialización del dominio de ventas consultivas.

### Hallazgos Críticos del Análisis Avanzado

4. **Calidad de Datos Excelente**: Solo **1 sentencia duplicada (0.17%)** y **0 problemas de codificación**, indicando alta calidad del corpus con **21 caracteres especiales únicos** bien manejados.

5. **Diversidad Léxica Variable**: TTR oscila desde **0.151** (Ejemplo de entregable al cliente - 5,235 palabras) hasta **0.667** (nuevo.txt - 6 palabras), con "Evaluación de la venta.txt" mostrando la mayor diversidad práctica (TTR=0.517).

6. **Desafíos de Legibilidad Críticos**: Score promedio de legibilidad de **7.48/100**, con **574 fragmentos (97.8%) clasificados como "difíciles"**, indicando contenido técnico altamente especializado que requiere estrategias específicas para IA.

7. **Distribución Heterogénea para FAISS**: Solo **295 fragmentos (50.3%) en rango óptimo** (68-213 caracteres), con **33 outliers (5.62%)** detectados y **26 fragmentos muy largos** (≥500 chars) que requieren fragmentación.